
# TA‑Lib Candlestick **Scanner + Advanced Confirmations** (v4 Extras)

This notebook scans daily OHLCV for candlestick patterns (TA‑Lib) and confirms them with a **toggleable** set of technical filters suitable for **swing trading**.

**New in this version**
- Added indicators & confirmations with **on/off toggles**: **ADX (DI+/DI−)**, **Aroon (Up/Down)**, **Stochastic (%K/%D)**, **Bollinger Bands** (with optional squeeze), **OBV**, **EOM**, and **ATR**.
- Keeps existing confirmations (RSI, MACD, Volume Spike, SMA‑200 trend).
- Outputs a CSV of **confirmed signals** with suggested **entry/stop (ATR‑based)** and **position size helper**.

> Note: This notebook downloads data with `yfinance` (internet required when you run it locally).



## 0) Setup
If TA‑Lib isn't installed, run the pip cell below. On Linux, you may need the system package `ta-lib` first.


In [1]:

# If needed, uncomment and run:
# %pip install TA-Lib yfinance pandas numpy matplotlib --quiet

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import datetime as dt

try:
    import talib as ta
except Exception as e:
    print("[WARN] TA-Lib not available. Install via: pip install TA-Lib")
    raise

try:
    import yfinance as yf
except Exception as e:
    print("[WARN] yfinance not available. Install via: pip install yfinance")
    raise

import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)



## 1) Configuration
- Add/remove tickers (Yahoo symbols; for NSE use `.NS` suffix).
- Toggle confirmations you want to enforce.
- `REQ_BULL_CONF` / `REQ_BEAR_CONF` defines the minimum number of confirmations required **among the toggled ones** (excluding the basic candlestick hit).


In [2]:

# ===== User Parameters =====
TICKERS = ['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'APLLTD.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'ARE&M.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AIIL.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BLS.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BERGEPAINT.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BSOFT.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'FIRSTCRY.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'MAPMYINDIA.NS', 'CCL.NS', 'CESC.NS', 'CGPOWER.NS', 'CRISIL.NS', 'CAMPUS.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CAPLIPOINT.NS', 'CGCL.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'CUB.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CAMS.NS', 'CONCORDBIO.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CROMPTON.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DCMSHRIRAM.NS', 'DLF.NS', 'DOMS.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'LALPATHLAB.NS', 'DRREDDY.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EICHERMOT.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'NYKAA.NS', 'FEDERALBNK.NS', 'FACT.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FSL.NS', 'FIVESTAR.NS', 'FORTIS.NS', 'GAIL.NS', 'GVT&D.NS', 'GMRAIRPORT.NS', 'GRSE.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'MEDANTA.NS', 'GODIGIT.NS', 'GPIL.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GESHIP.NS', 'FLUOROCHEM.NS', 'GUJGASLTD.NS', 'GMDCLTD.NS', 'GNFC.NS', 'GPPL.NS', 'GSPL.NS', 'HEG.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HEROMOTOCO.NS', 'HSCL.NS', 'HINDALCO.NS', 'HAL.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'POWERINDIA.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IFCI.NS', 'IIFL.NS', 'INOXINDIA.NS', 'IRB.NS', 'IRCON.NS', 'ITC.NS', 'ITI.NS', 'INDGN.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'IEX.NS', 'INDHOTEL.NS', 'IOC.NS', 'IOB.NS', 'IRCTC.NS', 'IRFC.NS', 'IREDA.NS', 'IGL.NS', 'INDUSTOWER.NS', 'INDUSINDBK.NS', 'NAUKRI.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'INDIGO.NS', 'IGIL.NS', 'IKS.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'JKCEMENT.NS', 'JBMA.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JPPOWER.NS', 'J&KBANK.NS', 'JINDALSAW.NS', 'JSL.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KPRMILL.NS', 'KEI.NS', 'KNRCON.NS', 'KPITTECH.NS', 'KAJARIACER.NS', 'KPIL.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KFINTECH.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KIMS.NS', 'LTF.NS', 'LTTS.NS', 'LICHSGFIN.NS', 'LTFOODS.NS', 'LTIM.NS', 'LT.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LUPIN.NS', 'MMTC.NS', 'MRF.NS', 'MGL.NS', 'MAHSEAMLES.NS', 'M&MFIN.NS', 'M&M.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MANKIND.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'METROPOLIS.NS', 'MINDACORP.NS', 'MSUMI.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NCC.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPCGREEN.NS', 'NTPC.NS', 'NH.NS', 'NATIONALUM.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NAM-INDIA.NS', 'NIVABUPA.NS', 'NUVAMA.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'PAYTM.NS', 'OFSS.NS', 'POLICYBZR.NS', 'PCBL.NS', 'PGEL.NS', 'PIIND.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'PTCIL.NS', 'PVRINOX.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFIZER.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PEL.NS', 'PPLPHARMA.NS', 'POLYMED.NS', 'POLYCAB.NS', 'POONAWALLA.NS', 'PFC.NS', 'POWERGRID.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PNB.NS', 'RRKABEL.NS', 'RBLBANK.NS', 'RECLTD.NS', 'RHIM.NS', 'RITES.NS', 'RADICO.NS', 'RVNL.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RKFORGE.NS', 'RCF.NS', 'RTNINDIA.NS', 'RAYMONDLSL.NS', 'RAYMOND.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RPOWER.NS', 'ROUTE.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SRF.NS', 'SAGILITY.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'MOTHERSON.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'RENUKA.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'STARHEALTH.NS', 'SBIN.NS', 'SAIL.NS', 'SWSOLAR.NS', 'SUMICHEM.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TBOTEK.NS', 'TVSMOTOR.NS', 'TANLA.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TTML.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'NIACL.NS', 'RAMCOCEM.NS', 'THERMAX.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TARIL.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRIVENI.NS', 'TRITURBINE.NS', 'TIINDIA.NS', 'UCOBANK.NS', 'UNOMINDA.NS', 'UPL.NS', 'UTIAMC.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UBL.NS', 'UNITDSPR.NS', 'USHAMART.NS', 'VGUARD.NS', 'DBREALTY.NS', 'VTL.NS', 'VBL.NS', 'MANYAVAR.NS', 'VEDL.NS', 'VIJAYA.NS', 'VMM.NS', 'IDEA.NS', 'VOLTAS.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZFCVINDIA.NS', 'ZEEL.NS', 'ZENTEC.NS', 'ZENSARTECH.NS', 'ZYDUSLIFE.NS', 'ECLERX.NS']


START = "2018-01-01"
END   = dt.date.today().isoformat()
INTERVAL = "1d"

# Patterns to scan (common set)
PATTERNS = [
    "CDLENGULFING","CDLPIERCING","CDLMORNINGSTAR","CDLEVENINGSTAR",
    "CDLDARKCLOUDCOVER","CDLHARAMI","CDLHARAMICROSS",
    "CDLHAMMER","CDLINVERTEDHAMMER","CDLSHOOTINGSTAR","CDLHANGINGMAN","CDLDOJI",
]

# ===== Toggleable Confirmations =====
# Momentum
USE_RSI = True
RSI_LEN = 14
RSI_LONG_MIN = 50     # long if RSI > 50
RSI_SHORT_MAX = 50    # short if RSI < 50

USE_MACD = True
MACD_FAST = 12; MACD_SLOW = 26; MACD_SIGNAL = 9
MACD_MODE = "line_cross"  # "line_cross" or "hist_above0"

USE_STOCH = True
STO_K = 14; STO_D = 3; STO_SLOW = 3
STO_MIN_LONG = 20     # prefer %K/%D > 20 for longs
STO_MAX_SHORT = 80    # prefer %K/%D < 80 for shorts
STO_REQUIRE_CROSS = True  # require %K cross above %D (long) or below (short)

# Trend strength / new-trend detectors
USE_ADX = True
ADX_LEN = 14
ADX_MIN = 25          # require ADX >=25 for strong trend
ADX_REQUIRE_DI_ALIGN = True  # long: DI+>DI-, short: DI->DI+

USE_AROON = True
AROON_LEN = 25
AROON_UP_MIN = 70     # long: AroonUp >= 70 and AroonUp > AroonDown
AROON_DN_MIN = 70     # short: AroonDown >= 70 and AroonDown > AroonUp

# Volatility bands
USE_BB = True
BB_LEN = 20; BB_N = 2.0
BB_REQUIRE_MID_BREAK = True        # long: Close >= mid; short: Close <= mid
BB_USE_SQUEEZE = False            # disable squeeze requirement
BB_SQUEEZE_PCTL = 0.30             # width in lowest 30% of last 120 bars triggers "squeeze"
BB_SQUEEZE_LOOKBACK = 120

# Volume-based confirmation
USE_VOL_SPIKE = True
VOL_LOOKBACK = 20
VOL_SPIKE_MULT = 1.5   # vol >= 1.5x 20d avg

USE_OBV = True         # On-Balance Volume
USE_EOM = False        # Ease of Movement disabled to broaden criteria

# Trend filter (location)
USE_TREND_FILTER = True
SMA_SHORT = 50
SMA_LONG  = 200       # long: Close >= SMA200; short: Close <= SMA200

# Volatility (ATR)
# Always computed for stops/sizing; optionally use as a filter to avoid ultra-low range
USE_ATR_FILTER = False
ATR_LEN = 14
ATR_MIN_PCT = 0.003   # lower threshold to 0.3%

# Confirmation quorum
REQ_BULL_CONF = 4
REQ_BEAR_CONF = 4
ALLOW_SHORTS = False

# Entry/stop suggestions
ENTRY_BUFFER_PCT = 0.003           # +0.3% over pattern high (long) or -0.3% under low (short)
STOP_ATR_MULT = 1.2
CAPITAL = 100000
RISK_PCT = 0.01

SAVE_CSV_PATH = "confirmed_candlestick_signals_talib_v4_extras.csv"



## 2) Helper functions
We compute indicators (TA‑Lib when available; EOM is implemented in pandas), run pattern detectors, and check confirmations per row.


In [3]:

def fetch_ohlcv(ticker: str, start: str, end: str, interval: str="1d") -> pd.DataFrame:
    df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, auto_adjust=False, multi_level_index=False)
    if df.empty:
        return df
    df.index = pd.to_datetime(df.index).tz_localize(None)

    o,h,l,c,v = df["Open"], df["High"], df["Low"], df["Close"], df["Volume"]

    # Core indicators
    df["RSI"] = ta.RSI(c, timeperiod=RSI_LEN) if USE_RSI else np.nan
    macd, macd_signal, macd_hist = ta.MACD(c, fastperiod=MACD_FAST, slowperiod=MACD_SLOW, signalperiod=MACD_SIGNAL) if USE_MACD else (np.nan, np.nan, np.nan)
    df["MACD"], df["MACD_SIGNAL"], df["MACD_HIST"] = macd, macd_signal, macd_hist
    if USE_STOCH:
        slowk, slowd = ta.STOCH(h, l, c, fastk_period=STO_K, slowk_period=STO_SLOW, slowd_period=STO_D)
        df["STO_K"], df["STO_D"] = slowk, slowd
        df["STO_K_prev"], df["STO_D_prev"] = df["STO_K"].shift(1), df["STO_D"].shift(1)
    else:
        df["STO_K"]=df["STO_D"]=df["STO_K_prev"]=df["STO_D_prev"]=np.nan

    if USE_ADX:
        df["ADX"] = ta.ADX(h,l,c, timeperiod=ADX_LEN)
        df["DI_PLUS"] = ta.PLUS_DI(h,l,c, timeperiod=ADX_LEN)
        df["DI_MINUS"]= ta.MINUS_DI(h,l,c, timeperiod=ADX_LEN)
    else:
        df["ADX"]=df["DI_PLUS"]=df["DI_MINUS"]=np.nan

    if USE_AROON:
        aroondown, aroonup = ta.AROON(h,l, timeperiod=AROON_LEN)
        df["AROON_UP"], df["AROON_DN"] = aroonup, aroondown
        df["AROON_OSC"] = aroonup - aroondown
    else:
        df["AROON_UP"]=df["AROON_DN"]=df["AROON_OSC"]=np.nan

    if USE_BB:
        upper, middle, lower = ta.BBANDS(c, timeperiod=BB_LEN, nbdevup=BB_N, nbdevdn=BB_N)
        df["BB_UPPER"], df["BB_MID"], df["BB_LOWER"] = upper, middle, lower
        df["BB_WIDTH"] = (upper - lower) / middle
        df["PCT_B"] = (c - lower) / (upper - lower)
        if BB_USE_SQUEEZE:
            win = max(BB_SQUEEZE_LOOKBACK, BB_LEN*6)
            rolling = df["BB_WIDTH"].rolling(win)
            thresh = rolling.quantile(BB_SQUEEZE_PCTL)
            df["BB_SQUEEZE"] = (df["BB_WIDTH"] <= thresh)
        else:
            df["BB_SQUEEZE"] = False
    else:
        for col in ["BB_UPPER","BB_MID","BB_LOWER","BB_WIDTH","PCT_B","BB_SQUEEZE"]:
            df[col] = np.nan

    if USE_OBV:
        df["OBV"] = ta.OBV(c, v)
        df["OBV_SLOPE"] = df["OBV"].diff()
    else:
        df["OBV"]=df["OBV_SLOPE"]=np.nan

    if USE_EOM:
        # Ease of Movement (basic implementation)
        mid_move = ((h + l)/2.0) - ((h.shift(1) + l.shift(1))/2.0)
        box_ratio = (h - l) / v.replace(0, np.nan)
        eom_raw = mid_move * box_ratio
        df["EOM14"] = eom_raw.rolling(14).mean()
    else:
        df["EOM14"] = np.nan

    # SMAs & ATR & Volume averages
    df["SMA50"] = ta.SMA(c, timeperiod=SMA_SHORT)
    df["SMA200"] = ta.SMA(c, timeperiod=SMA_LONG)
    df["ATR"] = ta.ATR(h,l,c, timeperiod=ATR_LEN)
    df["VOL20"] = v.rolling(VOL_LOOKBACK).mean()

    return df

def apply_patterns(df: pd.DataFrame, patterns: list) -> pd.DataFrame:
    out = []
    o,h,l,c = df["Open"], df["High"], df["Low"], df["Close"]
    base_cols = ["Open","High","Low","Close","Volume","RSI","MACD","MACD_SIGNAL","MACD_HIST",
                 "STO_K","STO_D","STO_K_prev","STO_D_prev",
                 "ADX","DI_PLUS","DI_MINUS","AROON_UP","AROON_DN","AROON_OSC",
                 "BB_UPPER","BB_MID","BB_LOWER","BB_WIDTH","PCT_B","BB_SQUEEZE",
                 "OBV","OBV_SLOPE","EOM14","SMA50","SMA200","ATR","VOL20"]
    for pname in patterns:
        if not hasattr(ta, pname):
            print(f"[WARN] TA-Lib has no function {pname}, skipping.")
            continue
        func = getattr(ta, pname)
        vals = func(o,h,l,c)
        sig = df.copy()[base_cols]
        sig["pattern"] = pname
        sig["raw_value"] = vals
        sig = sig[sig["raw_value"] != 0]
        if not sig.empty:
            out.append(sig)
    if not out:
        return pd.DataFrame()
    return pd.concat(out).sort_index()

def _meets_long(row, met: list):
    # RSI
    if USE_RSI and pd.notna(row["RSI"]) and row["RSI"] > RSI_LONG_MIN:
        met.append("RSI>50")
    # MACD
    if USE_MACD and pd.notna(row["MACD"]) and pd.notna(row["MACD_SIGNAL"]):
        if MACD_MODE == "line_cross":
            if row["MACD"] > row["MACD_SIGNAL"]:
                met.append("MACD>Signal")
        else:
            if row["MACD_HIST"] > 0:
                met.append("MACD_hist>0")
    # Stochastic
    if USE_STOCH and pd.notna(row["STO_K"]) and pd.notna(row["STO_D"]):
        cond_level = (row["STO_K"] > STO_MIN_LONG) or (row["STO_D"] > STO_MIN_LONG)
        cond_cross = True
        if STO_REQUIRE_CROSS and pd.notna(row["STO_K_prev"]) and pd.notna(row["STO_D_prev"]):
            cond_cross = (row["STO_K_prev"] <= row["STO_D_prev"]) and (row["STO_K"] > row["STO_D"])
        if cond_level and cond_cross:
            met.append("STO_K>D & >20")
    # ADX
    if USE_ADX and pd.notna(row["ADX"]):
        ok = (row["ADX"] >= ADX_MIN)
        if ADX_REQUIRE_DI_ALIGN and pd.notna(row["DI_PLUS"]) and pd.notna(row["DI_MINUS"]):
            ok = ok and (row["DI_PLUS"] > row["DI_MINUS"])
        if ok:
            met.append(f"ADX≥{ADX_MIN}")
    # Aroon
    if USE_AROON and pd.notna(row["AROON_UP"]) and pd.notna(row["AROON_DN"]):
        if row["AROON_UP"] >= AROON_UP_MIN and row["AROON_UP"] > row["AROON_DN"]:
            met.append("AroonUp↑")
    # Bollinger
    if USE_BB and pd.notna(row["BB_MID"]) and pd.notna(row["PCT_B"]):
        ok = True
        if BB_REQUIRE_MID_BREAK:
            ok = ok and (row["Close"] >= row["BB_MID"])
        if BB_USE_SQUEEZE and pd.notna(row["BB_SQUEEZE"]):
            ok = ok and bool(row["BB_SQUEEZE"])
        if ok:
            met.append("BB_conf")
    # Volume spike
    if USE_VOL_SPIKE and pd.notna(row["VOL20"]) and row["VOL20"] > 0:
        if row["Volume"] >= VOL_SPIKE_MULT * row["VOL20"]:
            met.append(f"Vol≥{VOL_SPIKE_MULT}x20d")
    # OBV
    if USE_OBV and pd.notna(row["OBV_SLOPE"]) and row["OBV_SLOPE"] > 0:
        met.append("OBV↑")
    # EOM
    if USE_EOM and pd.notna(row["EOM14"]) and row["EOM14"] > 0:
        met.append("EOM>0")
    # Trend filter
    if USE_TREND_FILTER and pd.notna(row["SMA200"]) and row["Close"] >= row["SMA200"]:
        met.append("Close≥SMA200")
    # ATR filter (optional)
    if USE_ATR_FILTER and pd.notna(row["ATR"]) and pd.notna(row["Close"]):
        if (row["ATR"] / max(row["Close"], 1e-9)) >= ATR_MIN_PCT:
            met.append(f"ATR≥{int(ATR_MIN_PCT*1000)/10}%")
    return met

def _meets_short(row, met: list):
    # RSI
    if USE_RSI and pd.notna(row["RSI"]) and row["RSI"] < RSI_SHORT_MAX:
        met.append("RSI<50")
    # MACD
    if USE_MACD and pd.notna(row["MACD"]) and pd.notna(row["MACD_SIGNAL"]):
        if MACD_MODE == "line_cross":
            if row["MACD"] < row["MACD_SIGNAL"]:
                met.append("MACD<Signal")
        else:
            if row["MACD_HIST"] < 0:
                met.append("MACD_hist<0")
    # Stochastic
    if USE_STOCH and pd.notna(row["STO_K"]) and pd.notna(row["STO_D"]):
        cond_level = (row["STO_K"] < STO_MAX_SHORT) or (row["STO_D"] < STO_MAX_SHORT)
        cond_cross = True
        if STO_REQUIRE_CROSS and pd.notna(row["STO_K_prev"]) and pd.notna(row["STO_D_prev"]):
            cond_cross = (row["STO_K_prev"] >= row["STO_D_prev"]) and (row["STO_K"] < row["STO_D"])
        if cond_level and cond_cross:
            met.append("STO_K<D & <80")
    # ADX
    if USE_ADX and pd.notna(row["ADX"]):
        ok = (row["ADX"] >= ADX_MIN)
        if ADX_REQUIRE_DI_ALIGN and pd.notna(row["DI_PLUS"]) and pd.notna(row["DI_MINUS"]):
            ok = ok and (row["DI_MINUS"] > row["DI_PLUS"])
        if ok:
            met.append(f"ADX≥{ADX_MIN}")
    # Aroon
    if USE_AROON and pd.notna(row["AROON_UP"]) and pd.notna(row["AROON_DN"]):
        if row["AROON_DN"] >= AROON_DN_MIN and row["AROON_DN"] > row["AROON_UP"]:
            met.append("AroonDn↑")
    # Bollinger
    if USE_BB and pd.notna(row["BB_MID"]) and pd.notna(row["PCT_B"]):
        ok = True
        if BB_REQUIRE_MID_BREAK:
            ok = ok and (row["Close"] <= row["BB_MID"])
        if BB_USE_SQUEEZE and pd.notna(row["BB_SQUEEZE"]):
            ok = ok and bool(row["BB_SQUEEZE"])
        if ok:
            met.append("BB_conf")
    # Volume spike
    if USE_VOL_SPIKE and pd.notna(row["VOL20"]) and row["VOL20"] > 0:
        if row["Volume"] >= VOL_SPIKE_MULT * row["VOL20"]:
            met.append(f"Vol≥{VOL_SPIKE_MULT}x20d")
    # OBV
    if USE_OBV and pd.notna(row["OBV_SLOPE"]) and row["OBV_SLOPE"] < 0:
        met.append("OBV↓")
    # EOM
    if USE_EOM and pd.notna(row["EOM14"]) and row["EOM14"] < 0:
        met.append("EOM<0")
    # Trend filter
    if USE_TREND_FILTER and pd.notna(row["SMA200"]) and row["Close"] <= row["SMA200"]:
        met.append("Close≤SMA200")
    # ATR filter
    if USE_ATR_FILTER and pd.notna(row["ATR"]) and pd.notna(row["Close"]):
        if (row["ATR"] / max(row["Close"], 1e-9)) >= ATR_MIN_PCT:
            met.append(f"ATR≥{int(ATR_MIN_PCT*1000)/10}%")
    return met

def confirm_signal(row: pd.Series, direction: str):
    met = []
    if direction == "bullish":
        met = _meets_long(row, met)
        req = REQ_BULL_CONF
    else:
        met = _meets_short(row, met)
        req = REQ_BEAR_CONF
    return (len(met) >= req, met)



## 3) Scan tickers
- Detect candlestick hits via TA‑Lib
- Apply chosen confirmations
- Suggest entries/stops and simple position size


In [4]:

all_signals = []

for t in TICKERS:
    print(f"Scanning {t} ...")
    df = fetch_ohlcv(t, START, END, INTERVAL)
    if df.empty:
        print(f"[WARN] No data for {t}")
        continue

    pats = apply_patterns(df, PATTERNS)
    if pats.empty:
        print(f"[INFO] No raw patterns for {t}")
        continue

    pats = pats.assign(ticker=t)
    rows = []
    for idx, r in pats.iterrows():
        direction = "bullish" if r["raw_value"] > 0 else "bearish"
        if direction == "bearish" and not ALLOW_SHORTS:
            continue

        ok, met = confirm_signal(r, direction)
        if not ok:
            continue

        # Entry/stop suggestions (ATR‑aware)
        if direction == "bullish":
            entry = r["High"] * (1 + ENTRY_BUFFER_PCT)
            base_stop = min(r["Low"], r["Close"])
            stop = base_stop - STOP_ATR_MULT * (r["ATR"] if pd.notna(r["ATR"]) else 0.0)
            note = "Buy on breakout > pattern High (+buffer)"
        else:
            entry = r["Low"] * (1 - ENTRY_BUFFER_PCT)
            base_stop = max(r["High"], r["Close"])
            stop = base_stop + STOP_ATR_MULT * (r["ATR"] if pd.notna(r["ATR"]) else 0.0)
            note = "Sell on breakdown < pattern Low (−buffer)"

        rps = abs(entry - stop)
        size = int((CAPITAL * RISK_PCT) // rps) if rps > 0 else 0

        rows.append({
            "ticker": t, "date": idx.date(), "pattern": r["pattern"], "direction": direction,
            # Core prices
            "open": round(r["Open"],4), "high": round(r["High"],4), "low": round(r["Low"],4), "close": round(r["Close"],4),
            # Momentum
            "rsi14": round(r["RSI"],2) if pd.notna(r["RSI"]) else np.nan,
            "macd": round(r["MACD"],4) if pd.notna(r["MACD"]) else np.nan,
            "macd_signal": round(r["MACD_SIGNAL"],4) if pd.notna(r["MACD_SIGNAL"]) else np.nan,
            "macd_hist": round(r["MACD_HIST"],4) if pd.notna(r["MACD_HIST"]) else np.nan,
            "sto_k": round(r["STO_K"],2) if pd.notna(r["STO_K"]) else np.nan,
            "sto_d": round(r["STO_D"],2) if pd.notna(r["STO_D"]) else np.nan,
            # Trend strength
            "adx": round(r["ADX"],2) if pd.notna(r["ADX"]) else np.nan,
            "di_plus": round(r["DI_PLUS"],2) if pd.notna(r["DI_PLUS"]) else np.nan,
            "di_minus": round(r["DI_MINUS"],2) if pd.notna(r["DI_MINUS"]) else np.nan,
            "aroon_up": round(r["AROON_UP"],1) if pd.notna(r["AROON_UP"]) else np.nan,
            "aroon_dn": round(r["AROON_DN"],1) if pd.notna(r["AROON_DN"]) else np.nan,
            "aroon_osc": round(r["AROON_OSC"],1) if pd.notna(r["AROON_OSC"]) else np.nan,
            # Bands & volatility
            "bb_upper": round(r["BB_UPPER"],4) if pd.notna(r["BB_UPPER"]) else np.nan,
            "bb_mid": round(r["BB_MID"],4) if pd.notna(r["BB_MID"]) else np.nan,
            "bb_lower": round(r["BB_LOWER"],4) if pd.notna(r["BB_LOWER"]) else np.nan,
            "bb_width": round(r["BB_WIDTH"],6) if pd.notna(r["BB_WIDTH"]) else np.nan,
            "pct_b": round(r["PCT_B"],4) if pd.notna(r["PCT_B"]) else np.nan,
            "bb_squeeze": bool(r["BB_SQUEEZE"]) if "BB_SQUEEZE" in r else False,
            "atr14": round(r["ATR"],4) if pd.notna(r["ATR"]) else np.nan,
            # Volume
            "vol": int(r["Volume"]), "vol20": int(r["VOL20"]) if pd.notna(r["VOL20"]) else np.nan,
            "obv": int(r["OBV"]) if pd.notna(r["OBV"]) else np.nan,
            "obv_slope": int(r["OBV_SLOPE"]) if pd.notna(r["OBV_SLOPE"]) else np.nan,
            "eom14": round(r["EOM14"],6) if pd.notna(r["EOM14"]) else np.nan,
            # Trend location
            "sma50": round(r["SMA50"],4) if pd.notna(r["SMA50"]) else np.nan,
            "sma200": round(r["SMA200"],4) if pd.notna(r["SMA200"]) else np.nan,
            # Meta
            "confirmations_met": ",".join(met), "num_conf": len(met),
            # Trading helpers
            "entry_suggest": round(entry,4), "stop_suggest": round(stop,4),
            "risk_per_share": round(rps,4), "size_for_1pct": int(size), "note": note
        })
    if rows:
        all_signals.append(pd.DataFrame(rows))

signals_df = pd.concat(all_signals).sort_values(["date","ticker"]).reset_index(drop=True) if all_signals else pd.DataFrame()
print(f"Found {len(signals_df)} confirmed signals.")
display(signals_df.tail(20))


Scanning 360ONE.NS ...
Scanning 3MINDIA.NS ...
Scanning ABB.NS ...
Scanning ACC.NS ...
Scanning ACMESOLAR.NS ...
Scanning AIAENG.NS ...
Scanning APLAPOLLO.NS ...
Scanning AUBANK.NS ...
Scanning AWL.NS ...
Scanning AADHARHFC.NS ...
Scanning AARTIIND.NS ...
Scanning AAVAS.NS ...
Scanning ABBOTINDIA.NS ...
Scanning ACE.NS ...
Scanning ADANIENSOL.NS ...
Scanning ADANIENT.NS ...
Scanning ADANIGREEN.NS ...
Scanning ADANIPORTS.NS ...
Scanning ADANIPOWER.NS ...
Scanning ATGL.NS ...
Scanning ABCAPITAL.NS ...
Scanning ABFRL.NS ...
Scanning ABREL.NS ...
Scanning ABSLAMC.NS ...
Scanning AEGISLOG.NS ...
Scanning AFCONS.NS ...
Scanning AFFLE.NS ...
Scanning AJANTPHARM.NS ...
Scanning AKUMS.NS ...
Scanning APLLTD.NS ...
Scanning ALIVUS.NS ...
Scanning ALKEM.NS ...
Scanning ALKYLAMINE.NS ...
Scanning ALOKINDS.NS ...
Scanning ARE&M.NS ...
Scanning AMBER.NS ...
Scanning AMBUJACEM.NS ...
Scanning ANANDRATHI.NS ...
Scanning ANANTRAJ.NS ...
Scanning ANGELONE.NS ...
Scanning APARINDS.NS ...
Scanning APOLLOH

,ticker,date,pattern,direction,open,high,low,close,rsi14,macd,macd_signal,macd_hist,sto_k,sto_d,adx,di_plus,di_minus,aroon_up,aroon_dn,aroon_osc,bb_upper,bb_mid,bb_lower,bb_width,pct_b,bb_squeeze,atr14,vol,vol20,obv,obv_slope,eom14,sma50,sma200,confirmations_met,num_conf,entry_suggest,stop_suggest,risk_per_share,size_for_1pct,note
116198,SBICARD.NS,2025-09-12,CDLENGULFING,bullish,851.55,863.4000,850.0500,856.1500,64.46,1.0952,-7.8652,8.9603,82.53,77.62,23.03,34.08,16.45,96.0,0.0,96.0,854.5033,816.4225,778.3417,0.093287,1.0216,False,18.6548,515577,773143,42627220,515577,NaN,840.9970,837.7667,"RSI>50,MACD>Signal,AroonUp↑,BB_conf,OBV↑,Close...",6,865.9902,827.6642,38.3260,26,Buy on breakout > pattern High (+buffer)
116199,SBILIFE.NS,2025-09-12,CDLENGULFING,bullish,1814.20,1834.5000,1803.6000,1830.2000,52.08,-5.0661,-5.7069,0.6408,46.92,39.17,16.71,28.07,14.78,24.0,84.0,-60.0,1876.4712,1825.5150,1774.5588,0.055827,0.5460,False,35.7881,509725,876376,190941455,509725,NaN,1826.1620,1635.6920,"RSI>50,MACD>Signal,BB_conf,OBV↑,Close≥SMA200",5,1840.0035,1760.6542,79.3493,12,Buy on breakout > pattern High (+buffer)
116200,SBIN.NS,2025-09-12,CDLDOJI,bullish,824.10,825.8000,819.8000,823.5500,60.61,1.4041,0.0207,1.3833,85.29,64.53,13.38,24.19,17.19,24.0,4.0,20.0,832.6526,814.3350,796.0174,0.044988,0.7515,False,8.8567,5078018,5311675,5228358744,-5078018,NaN,812.5800,789.4693,"RSI>50,MACD>Signal,BB_conf,Close≥SMA200",4,828.2774,809.1720,19.1054,52,Buy on breakout > pattern High (+buffer)
116201,SCHAEFFLER.NS,2025-09-12,CDLHAMMER,bullish,3969.80,3975.8999,3917.8999,3953.3000,52.18,-22.6870,-38.9391,16.2521,77.48,65.11,13.01,21.25,18.60,36.0,84.0,-48.0,4024.6325,3909.0500,3793.4676,0.059136,0.6914,False,89.7687,19374,48363,16457131,-19374,NaN,4020.1000,3645.8890,"RSI>50,MACD>Signal,BB_conf,Close≥SMA200",4,3987.8276,3810.1775,177.6501,5,Buy on breakout > pattern High (+buffer)
116202,SIEMENS.NS,2025-09-12,CDLDOJI,bullish,3202.00,3212.0000,3181.5000,3202.0000,56.00,13.3942,5.0889,8.3053,74.33,69.31,13.67,20.90,17.89,36.0,8.0,28.0,3245.3566,3138.5250,3031.6934,0.068078,0.7971,False,73.9055,172093,347985,124410509,172093,NaN,3130.6940,4404.5303,"RSI>50,MACD>Signal,BB_conf,OBV↑",4,3221.6360,3092.8134,128.8226,7,Buy on breakout > pattern High (+buffer)
116203,SKFINDIA.NS,2025-09-12,CDLHARAMICROSS,bullish,4829.00,4852.8999,4803.0000,4826.8999,59.68,52.9544,10.2451,42.7093,72.37,78.49,26.15,31.20,18.25,88.0,8.0,80.0,4962.9527,4633.7950,4304.6373,0.142068,0.7933,False,100.7716,8824,40573,10630264,8824,NaN,4743.0700,4378.9157,"RSI>50,MACD>Signal,ADX≥25,AroonUp↑,BB_conf,OBV...",7,4867.4586,4682.0741,185.3845,5,Buy on breakout > pattern High (+buffer)
116204,SKFINDIA.NS,2025-09-12,CDLHARAMI,bullish,4829.00,4852.8999,4803.0000,4826.8999,59.68,52.9544,10.2451,42.7093,72.37,78.49,26.15,31.20,18.25,88.0,8.0,80.0,4962.9527,4633.7950,4304.6373,0.142068,0.7933,False,100.7716,8824,40573,10630264,8824,NaN,4743.0700,4378.9157,"RSI>50,MACD>Signal,ADX≥25,AroonUp↑,BB_conf,OBV...",7,4867.4586,4682.0741,185.3845,5,Buy on breakout > pattern High (+buffer)
116205,SKFINDIA.NS,2025-09-12,CDLDOJI,bullish,4829.00,4852.8999,4803.0000,4826.8999,59.68,52.9544,10.2451,42.7093,72.37,78.49,26.15,31.20,18.25,88.0,8.0,80.0,4962.9527,4633.7950,4304.6373,0.142068,0.7933,False,100.7716,8824,40573,10630264,8824,NaN,4743.0700,4378.9157,"RSI>50,MACD>Signal,ADX≥25,AroonUp↑,BB_conf,OBV...",7,4867.4586,4682.0741,185.3845,5,Buy on breakout > pattern High (+buffer)
116206,STARHEALTH.NS,2025-09-12,CDLDOJI,bullish,443.50,445.9500,439.0500,443.3500,51.44,1.6072,2.0610,-0.4538,21.14,21.34,33.13,27.44,9.06,76.0,0.0,76.0,452.3382,442.7000,433.0618,0.043543,0.5337,False,13.9311,409120,960820,48190417,-409120,NaN,437.2300,427.0920,"RSI>50,ADX≥25,AroonUp↑,BB_conf,Close≥SMA200",5,447.2879,422.3327,24.9552,40,Buy on breakout > pattern High (+buffer)
116207,SYNGENE.NS,2025-09-12,CDLDOJI,bullish,664.10,666.8500,658.9000,664.1500,54.14,-2.6451,-4.8742,2.2290,75.25,60.72,16.79,21.56,25.12,28.0,64.0,-36.0,682.20


## 4) Save results


In [5]:

if not signals_df.empty:
    signals_df.to_csv(SAVE_CSV_PATH, index=False)
    print("Saved ->", SAVE_CSV_PATH)
else:
    print("[INFO] No confirmed signals. Consider loosening thresholds or toggles.")


Saved -> confirmed_candlestick_signals_talib_v4_extras.csv



## 5) Optional: quick plot helper


In [6]:

def plot_signal(ticker: str, date_str: str, window: int=80):
    df = fetch_ohlcv(ticker, START, END, INTERVAL)
    if df.empty:
        print("No data.")
        return
    d = pd.to_datetime(date_str)
    if d not in df.index:
        print("Date not found in OHLCV. Use a date from the CSV.")
        return
    i = df.index.get_loc(d)
    s = df.iloc[max(0, i-window): i+window]

    plt.figure(figsize=(12,5))
    plt.title(f"{ticker} around {date_str}")
    plt.plot(s.index, s["Close"], label="Close")
    if USE_BB:
        plt.plot(s.index, s["BB_UPPER"], label="BB Upper")
        plt.plot(s.index, s["BB_MID"],   label="BB Mid")
        plt.plot(s.index, s["BB_LOWER"], label="BB Lower")
    plt.plot(s.index, s["SMA50"], label="SMA50")
    plt.plot(s.index, s["SMA200"], label="SMA200")
    plt.legend(); plt.xlabel("Date"); plt.ylabel("Price")
    plt.show()

# Example:
# plot_signal("RELIANCE.NS", "2024-08-01")
